<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/rag_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup steps for pip install and copy db file**

In [1]:
!pip install rank_bm25
!pip install datasets
!pip install gradio
!pip install pymilvus pymilvus[model]
!pip install langchain_groq

from google.colab import drive
drive.mount('/content/drive')

!ls -l
!cp /content/drive/MyDrive/ragbench.db /content/ragbench.db
!ls -l

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import nltk
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from nltk.tokenize import sent_tokenize
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
from sentence_transformers import SentenceTransformer

nltk.download('punkt_tab')

# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = "BAAI/LLM-Embedder"
# embedding_model = "BAAI/bge-large-en"

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Tokenization and other utility functions**

In [3]:
# New code - 12/4 10 pm
import hashlib
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import scipy.sparse as sp

# Initialize the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(embedding_model)
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit=512, overlap=100):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

def process_document_with_identifiers(document):
    processed_data = []
    title_count = -1  # to start from 0
    # print("document>>>>>>>",document)
    for section in document:
        section_chunks = []
        passage_count = [ord('a')]  # Passage identifier as a list to handle nested increments
        title_count += 1  # Increment title count

        # Tokenize the section into sentences
        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                # New document detected
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"  # Identifier for the title

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count = [ord('a')]  # Reset passage count for the new document
            else:
                # Sentence under the current document
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                #print("chunked_texts>>>>process_document_with_identifiers>>>>> "+ "".join(chunked_texts))

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])

                # Increment passage_count intelligently
                i = len(passage_count) - 1
                while i >= 0:
                    passage_count[i] += 1
                    if passage_count[i] > ord('z'):
                        passage_count[i] = ord('a')
                        if i == 0:
                            passage_count.insert(0, ord('a'))  # Add a new character to the identifier
                        i -= 1
                    else:
                        break


        #print("section_chunks>>>>>>>",section_chunks)
        processed_data.append(section_chunks)

    return processed_data

# Function to generate a hash based on content and key metadata
def generate_hash(content, metadata):
    """Generate a unique hash for the document content and key metadata."""
    key_fields = f"{content}|{metadata.get('item_index')}|{metadata.get('prefix')}"
    return hashlib.md5(key_fields.encode('utf-8')).hexdigest()

# Function to retrieve existing hashes from the database
def get_existing_hashes(collection):
    """Retrieve all existing hashes (IDs) currently in the database."""
    all_records = collection.get(include=["documents", "metadatas"])  # Fetch documents and metadata
    existing_hashes = set()
    for doc, metadata in zip(all_records["documents"], all_records["metadatas"]):
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

# Function to retrieve existing hashes from the database
def get_existing_hashes_milvus(all_records):
    """Retrieve all existing hashes (IDs) currently in the database."""
    existing_hashes = set()
    print(f"all records >>> {len(all_records)}")
    if all_records == None or len(all_records) == 0:
        return existing_hashes

    for record in all_records:
        doc = record.get("documents")
        metadata = record.get("metadata")
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes


# Tokenize corpus and prepare BM25 encoder
def prepare_bm25_encoder(texts):
    tokenized_corpus = [word_tokenize(text.lower()) for text in texts]
    bm25_encoder = BM25Okapi(tokenized_corpus)
    return bm25_encoder

def generate_sparse_vector_bm25(query, bm25_encoder):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25_encoder.get_scores(tokenized_query)
    # Convert scores to CSR format
    sparse_vector = sp.csr_matrix(scores)
    return sparse_vector

## **Milvus VectorDataStore class**

In [4]:
import numpy as np
import time
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"

    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)


    def get_or_create_collection(self, name, vec_dim=128):
        try:
            self.get_collection(name)
        except:
            print(f"Collection {name} doesn't exist. Creating...")
            self.create_collection(name, vec_dim)


    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection_name = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        index_params.add_index(
            field_name="sparse",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="IP"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        self.current_collection = collection
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection

    def get_all_records(self, collection):
        all_records = self.client.query(
            collection_name=collection,
            filter=None,
            output_fields=["documents", "metadata"],
            limit=10000
        )
        if all_records == None:
            all_records = []

        return all_records

    def has_entities(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exists.")
        self.default_collection = name
        collection_stats = self.client.get_collection_stats(name)
        count = collection_stats.get("row_count", 0)  # Retrieve the number of entities
        return count

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], sparse_embs: np.ndarray, embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, sp_embs, emb, id in zip(metadata, documents, sparse_embs, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "sparse": sp_embs,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete(collection_name, expr="pk >= 0")
        self.client.flush([collection_name])

    def search(self, query_embedding: np.ndarray, collection='all', top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        if collection=='all':
          collections = self.client.list_collections()
        else:
          collections = [collection]

        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def sparse_search(self, query_embedding: np.ndarray, collection='all', top_k : int=10)-> list[dict[str, any]] :
        results = []
        if (collection=='all'):
          collections = self.client.list_collections()
        else:
          collections = [collection]
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "IP", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="sparse",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def hybrid_search(self, sparse_query_embedding: np.ndarray, dense_query_embedding: np.ndarray, collection='all', top_k : int=10, alpha=0.3)-> list[dict[str, any]] :
        results = []
        start_time = time.time()
        sparse_results = self.sparse_search(sparse_query_embedding, collection, top_k)
        n = int(len(sparse_results) * alpha)
        alpha_sparse_results = sparse_results[:n]
        dense_results = self.search(dense_query_embedding, collection, top_k)
        #'results = sorted(results, key=lambda x: x["distance"])[:top_k]
        results = dense_results + alpha_sparse_results
        end_time = time.time()
        print(f"Hybrid Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [result["documents"] for result in search_results if "documents" in result]

In [5]:
datasets = ['covidqa','cuad','delucionqa','emanual','expertqa','finqa','hagrid','hotpotqa','msmarco','pubmedqa','tatqa','techqa']
datastor = VectorDataStore()

insert_data = True
store_client = "Milvus"
num_records = 0

vector_dim = embedder.get_sentence_embedding_dimension()

# **Index Data into DB**

## **Don't run this everytime**

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import random

# Initialize storage for documents, IDs, and metadata

for i in range(12):
  all_documents = []
  all_ids = []
  all_metadatas = []

  # Process each dataset
  doc_idx = 0  # Global document index for unique IDs
  for dataset in [datasets[i]]:
      milvus_collection_name = f"ragbench_collection_{dataset}"
      try:
        num_records = datastor.has_entities(milvus_collection_name)
      except:
        num_records = 0
        print("we are good. collection doesnt exists")

      if num_records > 0:
          datastor.drop_collection(milvus_collection_name)

      datastor.get_or_create_collection(milvus_collection_name, vector_dim)

      print(f"Processing {dataset}...")
      data = load_dataset("rungalileo/ragbench", dataset, split="train")
      #only select first 5 records for debugging duplicate records. **PLEASE REMOVE THIS AFTER DEBUGGING**
      data = data.select(range(10))
      for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
          # Extract document text
          print(f"~~~~~~>>>  question: {row.get('question', '')}")
          doc_text = row.get('documents', '')

          # Skip if no documents found
          if not doc_text:
              continue

          # Process the document
          processed_output = process_document_with_identifiers(doc_text)
          added_item_idxs = set()

          # Populate the lists
          for section_idx, section in enumerate(processed_output):
              for item_idx, (prefix, content) in enumerate(section):
                  # Skip if this item_idx has already been processed
                  if item_idx in added_item_idxs:
                      continue

                  # Add the item_idx to the set to track it
                  added_item_idxs.add(item_idx)

                  # Add the document
                  document = f"[{prefix}] {content}"
                  all_documents.append(document)

                  # Construct a globally unique ID
                  doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                  all_ids.append(doc_id)

                  # Construct metadata
                  metadata = {
                      "dataset": dataset,
                      "global_index": doc_idx,
                      "section_index": section_idx,
                      "item_index": item_idx,
                      "prefix": prefix,
                      "type": "Title" if prefix.endswith("a") else "Passage",
                  }
                  all_metadatas.append(metadata)

          doc_idx += 1  # Increment global document index

  # Step 4: Generate Embeddings
  embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer
  batch_size = 2500  # Adjust based on available memory

  # Generate embeddings in batches
  all_embeddings = []
  for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
      batch_docs = all_documents[i:i + batch_size]
      batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
      all_embeddings.extend(batch_embeddings)

  bm25_encoder = prepare_bm25_encoder(all_documents)
  sparse_vectors = [generate_sparse_vector_bm25(text, bm25_encoder) for text in all_documents]

  # Adding data to milvus with enhanced duplicate check
  all_recs = datastor.get_all_records(milvus_collection_name)
  #print(f"sample: {str(all_recs[0])}")
  existing_hashes = get_existing_hashes_milvus(all_recs)

  for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
      batch_embeddings = all_embeddings[i:i + batch_size]
      batch_sparse_embs = sparse_vectors[i:i + batch_size]
      batch_metadatas = all_metadatas[i:i + batch_size]
      batch_documents = all_documents[i:i + batch_size]
      batch_ids = []

      # Generate hashes for each document in the batch
      for doc, metadata in zip(batch_documents, batch_metadatas):
          doc_hash = generate_hash(doc, metadata)
          if doc_hash not in existing_hashes:
              batch_ids.append(doc_hash)
              existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
          else:
              print(f"Skipping duplicate document: {doc[:15]}...")  # Print a preview of the duplicate doc

      # Add non-duplicate documents to the database
      if batch_ids:  # Ensure there are non-duplicate documents to add
          # Add the batch to the Milvus collection
          if store_client == "Milvus" and insert_data:
              datastor.insert(milvus_collection_name,
                  metadata=batch_metadatas,
                  documents=batch_documents,
                  sparse_embs = np.array(batch_sparse_embs),
                  embeddings=np.array(batch_embeddings),
                  ids=batch_ids
              )

  print(f"total data inserted into {milvus_collection_name} iteration {i}")
  print(f"records : {datastor.has_entities(milvus_collection_name)} ")

#**Retrieval part**

## retrieval function declarations

In [6]:
import torch
import pandas as pd
import langchain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Function to retrieve relevant chunks
def retrieve_docs_milvus(question, collection_name, top_k=10):
    # Generate embedding for the query
    # Generate the dense embedding for the question
    start_time = time.time()
    dense_query_embedding = embedder.encode(question).tolist()

    # Generate the sparse embedding for the question
    #sparse_query_embedding = generate_sparse_vector_bm25(question, bm25_encoder)

    # Perform vector search to find relevant chunks
    #results = datastor.extract_documents(datastor.search(query_embedding, milvus_collection_name, top_k))
    print(f"{collection_name} has {datastor.has_entities(collection_name)} records")
    results = datastor.search(dense_query_embedding, collection_name, top_k)
    #results = datastor.hybrid_search(sparse_query_embedding, dense_query_embedding, milvus_collection_name, top_k)
    print(f"results: hybrid_search >>>  {results} from {collection_name}")

    # HyDE search with pseudo document
    #pseudo_docs = fetch_docs_pseudo(query)
    #print(f"pseudo_docs: retrieve_docs_milvus >>>  {pseudo_docs}")

    # Extract 'documents' field
    documents_list = [item['documents'] for item in results]
    #documents_list += pseudo_docs

    end_time = time.time()
    print(f"Hybrid Search completed. Found {len(results)} results (approx. 2 * {top_k} + alpha * {top_k}). in {end_time - start_time} secs")
    # Print the extracted documents
    print("retrieve_docs_milvus >>> documents_list from Hybrid + HyDE search >>>>", documents_list)

    # Extract the retrieved chunks
    # chunks = documents_list
    # should sort and push context - but later

    return documents_list

# Load Cross-Encoder model and tokenizer
cross_encoder_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
tokenizer = AutoTokenizer.from_pretrained(cross_encoder_model_name)
model = AutoModelForSequenceClassification.from_pretrained(cross_encoder_model_name)

# Ensure PyTorch is in evaluation mode
model.eval()

def rerank_with_cross_encoder(question, collection_name):
    """
    Rerank documents based on relevance scores from a Cross-Encoder model.

    Args:
        query (str): The query string.
        documents (list): A list of document strings.

    Returns:
        list: A list of tuples (document, score), sorted by score in descending order.
    """
    query_embedding = embedder.encode(question)
    # results = datastor.search(query_embedding, collection_name, 5)
    results = datastor.hybrid(query_embedding, collection_name, 10)
    documents = datastor.extract_documents(results)

    scores = []
    for doc in documents:
        # Tokenize query-document pair
        inputs = tokenizer(
            question,
            doc,
            return_tensors="pt",
            max_length=512,  # Limit for most transformer models
            truncation=True,
            padding="max_length",
        )
        # Compute relevance scores
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Handle binary classification or regression logits
            if logits.size(1) == 2:  # Binary classification
                score = torch.softmax(logits, dim=1)[:, 1].item()  # Probability of relevance (class 1)
            else:  # Regression or single-class output
                score = logits.squeeze().item()  # Direct score (e.g., relevance regression)

            scores.append((doc, score))
    # Sort by score in descending order
    return sorted(scores, key=lambda x: x[1], reverse=True)

def query_response_from_llm(query: str, collection_name, model_name="llama3-8b-8192"):

    # retrieve chunks from milvus db
    chunks = retrieve_docs_milvus(query, collection_name)

    if not chunks:
        return "No relevant documents found.", ""

    # retrieve chunks from chroma db
    #chunks = retrieve_docs(query)

    # Flatten the list if necessary
    if any(isinstance(chunk, list) for chunk in chunks):
      chunks = [item for sublist in chunks for item in (sublist if isinstance(sublist, list) else [sublist])]

    chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name=model_name)

    #chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="deepseek-r1-distill-llama-70b")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )


    chain = prompt | chat

    context = "".join(chunks)

    print("context>>>from 1st RAG>>>>>> ",context)

    groq_response = chain.invoke({"context": context, "query": query})

    print("groq_response>>>from 1st RAG>>>>>> ",groq_response.content)

    answer = groq_response.content
    return answer, context

def generate_prompt():
    """
    Generate a prompt template for assessing the support and relevance of an LLM-generated response.
    """
    return """
    I asked someone to answer a question based on one or more documents.
    Your task is to review their response and assess whether or not each sentence
    in that response is supported by text in the documents. And if so, which
    sentences in the documents provide that support. You will also tell me which
    of the documents contain useful information for answering the question, and
    which of the documents the answer was sourced from.
    Here are the documents, each of which is split into sentences.Alongside each
    sentence is associated key, such as ’[0a].’ or ’[0b].’ that you can use to refer
    to it:

    ‘‘‘
    {documents}
    ‘‘‘
    The question was:
    ‘‘‘
    {question}
    ‘‘‘

    Here is their response, split into sentences. Alongside each sentence is
    associated key, such as ’a.’ or ’b.’ that you can use to refer to it. Note
    that these keys are unique to the response, and are not related to the keys
    in the documents:
    ‘‘‘
    {answer}
    ‘‘‘
    You must respond with a JSON object matching this schema:
    ‘‘‘
    {{
    "relevance_explanation": string,
    "all_relevant_sentence_keys": [string],
    "overall_supported_explanation": string,
    "overall_supported": boolean,
    "sentence_support_information": [
    {{
    "response_sentence_key": string,
    "explanation": string,
    "supporting_sentence_keys": [string],
    "fully_supported": boolean
    }},
    ],
    "all_utilized_sentence_keys": [string]
    }}
    ‘‘‘
    The relevance_explanation field is a string explaining which documents
    contain useful information for answering the question. Provide a step-by-step
    breakdown of information provided in the documents and how it is useful for
    answering the question.
    The all_relevant_sentence_keys field is a list of all document sentences keys
    (e.g. ’0a’) that are relevant to the question. Include every sentence that is
    useful and relevant to the question, even if it was not used in the response,
    or if only parts of the sentence are useful. Ignore the provided response when
    making this judgement and base your judgement solely on the provided documents
    and question. Omit sentences that, if removed from the document, would not
    impact someone’s ability to answer the question.
    The overall_supported_explanation field is a string explaining why the response
    *as a whole* is or is not supported by the documents. In this field, provide a
    step-by-step breakdown of the claims made in the response and the support (or
    lack thereof) for those claims in the documents. Begin by assessing each claim
    separately, one by one; don’t make any remarks about the response as a whole
    until you have assessed all the claims in isolation.
    The overall_supported field is a boolean indicating whether the response as a
    whole is supported by the documents. This value should reflect the conclusion
    you drew at the end of your step-by-step breakdown in overall_supported_explanation.
    In the sentence_support_information field, provide information about the support
    *for each sentence* in the response.
    The sentence_support_information field is a list of objects, one for each sentence
    in the response. Each object MUST have the following fields:
    - response_sentence_key: a string identifying the sentence in the response.
    This key is the same as the one used in the response above.

    - explanation: a string explaining why the sentence is or is not supported by the
    documents.
    - supporting_sentence_keys: keys (e.g. ’[0a]’) of sentences from the documents that
    support the response sentence. If the sentence is not supported, this list MUST
    be empty. If the sentence is supported, this list MUST contain one or more keys.
    In special cases where the sentence is supported, but not by any specific sentence,
    you can use the string "supported_without_sentence" to indicate that the sentence
    is generally supported by the documents. Consider cases where the sentence is
    expressing inability to answer the question due to lack of relevant information in
    the provided context as "supported_without_sentence". In cases where the sentence
    is making a general statement (e.g. outlining the steps to produce an answer, or
    summarizing previously stated sentences, or a transition sentence), use the
    string "general". In cases where the sentence is correctly stating a well-known fact,
    like a mathematical formula, use the string "well_known_fact". In cases where the
    sentence is performing numerical reasoning (e.g. addition, multiplication), use
    the string "numerical_reasoning".
    - fully_supported: a boolean indicating whether the sentence is fully supported by
    the documents.
    - This value should reflect the conclusion you drew at the end of your step-by-step
    breakdown in explanation.
    - If supporting_sentence_keys is an empty list, then fully_supported must be false.
    - Otherwise, use fully_supported to clarify whether everything in the response
    sentence is fully supported by the document text indicated in supporting_sentence_keys
    (fully_supported = true), or whether the sentence is only partially or incompletely
    supported by that document text (fully_supported = false).
    The all_utilized_sentence_keys field is a list of all sentences keys (e.g. ’0a’) that
    were used to construct the answer. Include every sentence that either directly supported
    the answer, or was implicitly used to construct the answer, even if it was not used
    in its entirety. Omit sentences that were not used, and could have been removed from
    the documents without affecting the answer.
    You must respond with a valid JSON string. Use escapes for quotes, e.g. \\"\\", and
    newlines, e.g. \\n. Do not write anything before or after the JSON string. Do not
    wrap the JSON string in backticks like ‘‘‘ or ‘‘‘json.
    As a reminder: your task is to review the response and assess which documents contain
    useful information pertaining to the question, and how each sentence in the response
    is supported by the text in the documents.
    """.strip()

def analyze_llm_response(question, context, answer):
  chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-70b-8192")

  prompt_template_with_docs = PromptTemplate(
      input_variables=["documents", "question", "answer"],
      template=generate_prompt(),
  )

  chain = prompt_template_with_docs | chat
  groq_response_with_context_qanda = chain.invoke({"documents": context, "question": question, "answer":answer})
  return groq_response_with_context_qanda

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [7]:
import re
from sklearn.metrics import mean_squared_error, roc_auc_score
import numpy as np
from sklearn.metrics import roc_auc_score
from datasets import load_dataset
from datasets import concatenate_datasets

# Helper function for length computation (mocked as sentence count here)
def compute_length(keys):
    return len(keys)

# Metrics Computation
def compute_metrics(data):
    all_relevant = data["all_relevant_sentence_keys"]
    all_utilized = data["all_utilized_sentence_keys"]
    sentences_info = data["sentence_support_information"]

    # Context Relevance
    total_relevant_length = compute_length(all_relevant)
    total_context_length = total_relevant_length  # Assuming all relevant are part of the context
    context_relevance = total_relevant_length / total_context_length if total_context_length > 0 else 0

    # Context Utilization
    total_utilized_length = compute_length(all_utilized)
    context_utilization = total_utilized_length / total_context_length if total_context_length > 0 else 0

    # Completeness
    total_relevant_utilized = sum(
        1 for s in sentences_info if set(s["supporting_sentence_keys"]).intersection(all_utilized)
    )
    completeness = total_relevant_utilized / total_relevant_length if total_relevant_length > 0 else 0

    # Adherence
    adherence = data['overall_supported']

    return {
        "Context Relevance": context_relevance,
        "Context Utilization": context_utilization,
        "Completeness": completeness,
        "Adherence": adherence
    }


# Compute and print metrics
def fetch_ground_truth(dataset_name, question):
    dataset_dict = load_dataset("rungalileo/ragbench", dataset_name)
    dataset = concatenate_datasets([ds for ds in dataset_dict.values()])  # Combine all splits

    for sample in dataset:  # Iterate over the concatenated dataset
        if sample["question"] == question:
            return {
                "Context Relevance": sample.get("relevance_score"),  # Use .get() to avoid KeyErrors
                "Context Utilization": sample.get("utilization_score"),
                "Adherence": sample.get("adherence_score")
            }
    return None  # Ensure None is returned only if no match is found


def compute_rmse(predicted, ground_truth):
    # Extract true and predicted values
    y_true_relevance = ground_truth["Context Relevance"]
    y_true_utilization = ground_truth["Context Utilization"]

    y_pred_relevance = predicted["Context Relevance"]
    y_pred_utilization = predicted["Context Utilization"]

    # Compute RMSE for Context Relevance and Context Utilization
    rmse_relevance = np.sqrt((y_pred_relevance - y_true_relevance) ** 2)
    rmse_utilization = np.sqrt((y_pred_utilization - y_true_utilization) ** 2)

    return {
        "RMSE-Relevance": rmse_relevance,
        "RMSE-Utililization": rmse_utilization,
    }


def compute_auc_roc(predicted_adherence, ground_truth_adherence):
    # Handle boolean values directly
    y_true_numeric_alt = [1 if item else 0 for item in ground_truth_adherence]
    y_pred_numeric_alt = [1 if item else 0 for item in predicted_adherence]

    # Compute AUC-ROC
    auc_roc = roc_auc_score(y_true_numeric_alt, y_pred_numeric_alt)
    return auc_roc

def compute_metrics_rmse_roc(data, dataset_name, question):
    predicted_metrics = compute_metrics(data)
    print(f"query >>>> {question}")
    print(f"dataset_name >>>> {dataset_name}")
    ground_truth = fetch_ground_truth(dataset_name, question)
    print(f"ground truth {json.dumps(ground_truth)}")
    print(f"predict truth {json.dumps(predicted_metrics)}")

    rmse_values = compute_rmse(predicted_metrics, ground_truth)
    auc_roc = None
    if ground_truth:
      ground_truth_adherence = ground_truth['Adherence']
      predicted_adherence = predicted_metrics["Adherence"]
      auc_roc = compute_auc_roc(predicted_adherence,ground_truth_adherence)

    return {
        "RMSE-Relevance": rmse_values["RMSE-Relevance"],
        "RMSE-Utililization": rmse_values["RMSE-Utililization"],
        "AUC-ROC": auc_roc
    }


In [9]:
# Added for testing the fix for fetch_ground_truth. Remove after testing

from datasets import load_dataset
from datasets import concatenate_datasets
import json
# Compute and print metrics
def fetch_ground_truth(dataset_name, question):
    dataset_dict = load_dataset("rungalileo/ragbench", dataset_name, split=None)
    dataset = concatenate_datasets([ds for ds in dataset_dict.values()])  # Combine all splits

    for sample in dataset:  # Iterate over the concatenated dataset
        if sample["question"] == question:
            return {
                "Context Relevance": sample.get("relevance_score"),  # Use .get() to avoid KeyErrors
                "Context Utilization": sample.get("utilization_score"),
                "Adherence": sample.get("adherence_score")
            }
    return None  # Ensure None is returned only if no match is found

question = "What have sero-surveys of MERS virus found?"
ground_truth = fetch_ground_truth("covidqa", question)
print(f"ground truth {json.dumps(ground_truth)}")

README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/854k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/913k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1252 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/246 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/267 [00:00<?, ? examples/s]

ground truth {"Context Relevance": 0.17391304347826086, "Context Utilization": 0.13043478260869565, "Adherence": false}


## **Setup function for Gradio**

In [10]:
import json

# Placeholder function to get response from your LLM/RAG pipeline
def get_response(query, model_name, rag_bench):
    collection_name = f"ragbench_collection_{rag_bench}"
    print(f"Response from {collection_name} for query: {query}")  # Replace with actual model call
    answer, context = query_response_from_llm(query, collection_name, model_name)
    if not context:
      return answer, "unable to generate stats from llm"
    response = analyze_llm_response(query, context, answer)
    data = json.loads(response.content)
    predict_metrics = compute_metrics(data)
    ground_truth = fetch_ground_truth(rag_bench, query)
    if not ground_truth:
      return answer, "" + json.dumps(data, indent=4) + "```"
    analytics_json = compute_rmse(predict_metrics, ground_truth)
    stats = "```" + json.dumps({"data": data, "analytics_json": analytics_json}, indent=4) + "```"
    return answer, stats


In [11]:
answer, resp = get_response("What have sero-surveys of MERS virus found?", "llama3-8b-8192", 'covidqa')
print(answer)
print(resp)

Response from ragbench_collection_covidqa for query: What have sero-surveys of MERS virus found?
ragbench_collection_covidqa has 57 records
search results size : 1
Collection: ragbench_collection_covidqa, data: {'id': '68c0015df08449fcd270c34d2455994b', 'distance': 0.9065372347831726, 'entity': {'documents': '[0a] Title: MERS coronavirus: diagnostics, epidemiology and transmission\nPassage: for human sero-surveys.', 'metadata': {'dataset': 'covidqa', 'global_index': 2, 'section_index': 0, 'item_index': 0, 'prefix': '0a', 'type': 'Title'}}}
Collection: ragbench_collection_covidqa, data: {'id': 'fb017e0b19894d6f7996613624e91541', 'distance': 0.8914117217063904, 'entity': {'documents': '[0b] No sign of MERS-CoV antibodies was found among 2,400 sera from patients visiting Hospital in Jeddah, from 2010 through 2012, prior to the description of MERS-CoV .', 'metadata': {'dataset': 'covidqa', 'global_index': 2, 'section_index': 0, 'item_index': 2, 'prefix': '0b', 'type': 'Passage'}}}
Collecti

In [12]:
import gradio as gr


# Available LLM models
llm_models = ["llama3-8b-8192", "deepseek-r1-distill-llama-70b"]  # Modify as needed
rag_banch_datasets = ['covidqa','cuad','delucionqa','emanual','expertqa',
                      'finqa','hagrid','hotpotqa','msmarco','pubmedqa','tatqa','techqa']

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## LLM/RAG Query Interface")

    with gr.Row():
        query_input = gr.Textbox(label="Enter your query")
        rag_bench_selector = gr.Dropdown(rag_banch_datasets, label="Select RAG Bench Dataset")
        model_selector = gr.Dropdown(llm_models, label="Select LLM Model")

    submit_btn = gr.Button("Submit")
    answer = gr.Textbox(label="LLM Response", interactive=False)
    res_json = gr.Code(label="LLM Response Evaluation", interactive=False, language="json")

    submit_btn.click(get_response, inputs=[query_input, model_selector, rag_bench_selector], outputs=(answer, res_json))

# Launch the interface
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://14b15fca1852a5c9e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Response from ragbench_collection_techqa for query: JD Edwards Adapter supported in BPM 8.5.6? Hi experts, I have a question about the support for the JD Edwards Adapter in BPM 8.5.6:  which is the most current version of JD Edwards supported in this Adapter ?
ragbench_collection_techqa has 280 records
search results size : 1
Collection: ragbench_collection_techqa, data: {'id': 'ea534f288a26a3b30ebd93e71781948f', 'distance': 0.8435539603233337, 'entity': {'documents': '[0w] [/support/docview.wss?uid=swg27048530&amp;aid=5] Back to Top \n\n\n\nRelated information: \n * Update Strategy for WebSphere Application Server [http://www.ibm.com/support/docview.wss?uid=swg27036014] \n   \n * Fix list for IBM WebSphere Application Server Liberty [http://www.ibm.com/support/docview.wss?uid=swg27043863]\n   \n * CICS TS embedded components [http://www.ibm.com/support/knowledgecenter/SSGMCP_5.3.0/com.ibm.cics.ts.standards.doc/topics/supportedComponents.html]\n   \n * Fixes by version for CICS product

train-00000-of-00001.parquet:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.40M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1192 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/304 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/314 [00:00<?, ? examples/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://14b15fca1852a5c9e8.gradio.live
